In [1]:
import matplotlib.pyplot as plt
import numpy as np
colors = [{1:'black', -1:'gray'},{1:'darkred', -1:'pink'}]


class Marker:
    def __init__(self, type : int, subtype : int, time : int, value : int):
        self.type = type
        self.subtype = subtype
        self.time = time
        self.value = value

def plot_markers(markers, J):
    plt.vlines([marker.time for marker in markers],J[0]/2 + 10,J[1]/2 - 10, colors = [colors[marker.subtype][marker.type] for marker in markers])

def plot_bounds(datum,J):
    datum = datum[1:]
    start, end = datum[0], datum[1]
    plt.plot()
    plt.vlines([start,end],J[0]/2 + 10,J[1]/2 - 10, colors = ['red', 'black'])
    plt.hlines(J[0]/4 + J[1]/4, start, end)

In [2]:

def plot_gaussian(datum):
    mu = datum[0]
    A = datum[1]
    sigma = datum[2]
    x = np.arange(mu - 100, mu + 100, 0.1)
    y = A*np.exp(-0.5*np.square(x - mu)/(sigma*sigma))
    plt.plot(x, y, linestyle = "dashed")

def plot_biexp_curve(t, kappa, rho):
    return np.exp(-t/kappa) - np.exp(-t/rho)

def plot_biexp(datum):
    datum = datum[1:]
    
    start, end = datum[0], datum[1]
    baseline, amp, kappa, rho = datum[2], datum[3], datum[4], datum[5]
    x = np.arange(start, end, 0.1)
    y = amp*plot_biexp_curve(x - start, kappa, rho) + baseline
    plt.plot(x, y, linestyle = "dashed")

def plot_second_order_cpx(x, A,B,C, param_lambda, param_theta):

    return np.exp(-x*param_lambda)*(A*np.cos(0.5*x*param_theta) + B*np.sin(0.5*x*param_theta)) + C

def plot_second_order(datum):
    datum = datum[1:]
    start, end, _ = datum[0], datum[1], datum[2]
    param_lambda, param_theta = datum[3], datum[4]
    A,B,C = datum[5], datum[6], datum[7]
    x = np.arange(start, end, 0.1)
    y = plot_second_order_cpx(x - start, A, B, C, param_lambda, param_theta)
    plt.plot(x,y, linestyle = "dashed")

In [3]:


# f = open('data/retrace.csv')
# tokens = [e.split(",") for e in f.readlines()]
# x = [int(e[0]) for e in tokens]
# y = [int(e[1]) for e in tokens]
# handle = plt.plot(x,y)

def get_trace_file(filename):
    f = open(filename, 'r')
    tokens = [[float(x) for x in e.split(",")] for e in f.readlines()]
    #tokens = [token for token in tokens if I[0] <= token[0] <= I[1]]
    #maxy = max([e[1] for e in tokens])
    x = [e[0] for e in tokens]
    y = [e[1] for e in tokens]
    return (x,y)

def get_detect_file(filename):
    f = open(filename, 'r')
    tokens = [[float(x) for x in e.split(",")] for e in f.readlines()]
    markers = [Marker(int(token[1]),int(token[2]),int(token[0]),int(token[3])) for token in tokens]
    return markers

def get_event_file(filename):
    f = open(filename, 'r')
    tokens = [[float(x) for x in e.split(",")] for e in f.readlines()]
    return tokens


In [4]:
def create_trace_plot(ax,data,I,J):
    plt.subplot(ax)
    plt.xlim(I)
    #plt.ylim(J)
    plt.plot(data[0],data[1])

def create_event_plot(ax,data,I,J):
    plt.subplot(ax)
    plt.xlim(I)
    plt.ylim(J)
    plt.subplot(ax)
    for datum in data:
        plot_bounds(datum,J)
        plot_second_order(datum)

In [5]:

def create_plot(data, I):
    fig = plt.figure(figsize = (14,12))
    maxy = max(data[0][1]) + 10
    ax = plt.subplot(2,1,1)
    create_trace_plot(ax,data[0],I,(0,maxy))
    create_trace_plot(ax,data[1],I,(0,maxy))
    #create_trace_plot(ax,data[2],I,(0,maxy))
    #create_trace_plot(ax,data[2],I,(0,maxy))
    plot_markers(data[5],(0,maxy))

    return fig,ax

In [6]:
from ipywidgets import interact, widgets

data = [get_trace_file('Saves/output_baselined.csv'),
        get_trace_file('Saves/output_smoothed.csv'),
        get_trace_file('Saves/output_diff.csv'),
        get_trace_file('Saves/output_cuts.csv'),
        get_trace_file('Saves/output_simulated.csv'),
        get_detect_file('Saves/output_pulses.csv')]

def func(delta, size):
    I = (delta,delta + size)
    fig,ax = create_plot(data,I)

interact(func,
    delta = widgets.IntSlider(min=0, max=30000, step=100, value=3000),
    size = widgets.IntSlider(min=100, max=25000, step=100, value=500)
)

interactive(children=(IntSlider(value=3000, description='delta', max=30000, step=100), IntSlider(value=100, de…

<function __main__.func(delta, size)>